In [1]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn

import argparse
import time
import pandas as pd

from easydict import EasyDict as edict
from tqdm import trange
from torchsummary import summary
from torchstat import stat

YOUR_PATH = "/home/jyt/workspace/fNIRS_models/code_data_tufts"
sys.path.insert(0, YOUR_PATH + '/fNIRS-mental-workload-classifiers/helpers')
import models
import brain_data
from utils import generic_GetTrainValTestSubjects, seed_everything, makedir_if_not_exist, plot_confusion_matrix, save_pickle, train_one_epoch, eval_model, save_training_curves_FixedTrainValSplit, write_performance_info_FixedTrainValSplit, write_program_time, write_inference_time
from utils import LabelSmoothing, train_one_epoch_fNIRS_T, eval_model_fNIRST, train_one_epoch_Ours_T, eval_model_OursT
from utils import EarlyStopping

OursT parameters

In [2]:
cuda = torch.cuda.is_available()
if cuda:
    print('Detected GPUs', flush = True)
    #device = torch.device('cuda')
    device = torch.device('cuda:{}'.format(0))
else:
    print('DID NOT detect GPUs', flush = True)
    device = torch.device('cpu')

Detected GPUs


In [3]:
model_to_use = models.Ours_T
model = model_to_use(n_class=2, sampling_points=150, patch_length=30, dim=64, depth=6, heads=8, mlp_dim=256).to(device)

In [4]:
summary(model, input_size=(150, 8), batch_size=-1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Rearrange-1               [-1, 8, 150]               0
         Rearrange-2             [-1, 8, 5, 30]               0
         Rearrange-3             [-1, 5, 8, 30]               0
         Rearrange-4               [-1, 5, 240]               0
            Linear-5                [-1, 5, 64]          15,424
           Dropout-6                [-1, 6, 64]               0
         LayerNorm-7                [-1, 6, 64]             128
            Linear-8              [-1, 6, 1536]          98,304
            Linear-9                [-1, 6, 64]          32,832
          Dropout-10                [-1, 6, 64]               0
        Attention-11                [-1, 6, 64]               0
          PreNorm-12                [-1, 6, 64]               0
         Residual-13                [-1, 6, 64]               0
        LayerNorm-14                [-1

In [5]:
model_to_use = models.fNIRS_PreT
model = model_to_use(n_class=2, sampling_point=300, dim=64, depth=6, heads=8, mlp_dim=64).to(device)

In [6]:
summary(model, input_size=(2,2,300), batch_size=-1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         AvgPool1d-1               [-1, 2, 300]               0
         AvgPool1d-2               [-1, 2, 300]               0
         AvgPool1d-3               [-1, 2, 300]               0
         LayerNorm-4               [-1, 2, 300]             600
         AvgPool1d-5               [-1, 2, 300]               0
         AvgPool1d-6               [-1, 2, 300]               0
         AvgPool1d-7               [-1, 2, 300]               0
         LayerNorm-8               [-1, 2, 300]             600
          PreBlock-9            [-1, 2, 2, 300]               0
           Conv2d-10             [-1, 8, 3, 91]             968
        Rearrange-11               [-1, 3, 728]               0
           Linear-12                [-1, 3, 64]          46,656
        LayerNorm-13                [-1, 3, 64]             128
           Conv2d-14             [-1, 8